# Влияние эффекта интерференции на качество jpg изображений и рассчет коэффициентов MSE PSNR и SSIM

## Получаем исходные изображения

In [1]:
from IPython.display import Image, display, HTML

def make_html(img_path):
     return '<img src="{}" style="display:inline;margin:10px;" height="128"/>'.format(img_path)

_group_1_paths = ['_images/group_1/_1.jpg', '_images/group_1/_2.jpg', '_images/group_1/_3.jpg']
_group_2_paths = ['_images/group_2/_1.jpg', '_images/group_2/_2.jpg', '_images/group_2/_3.jpg']
_group_3_paths = ['_images/group_3/_1.jpg', '_images/group_3/_2.jpg', '_images/group_3/_3.jpg']

print("Малое количество деталей")
display(HTML(''.join([make_html(x) for x in _group_1_paths])))

print("Средне количество деталей")
display(HTML(''.join([make_html(x) for x in _group_2_paths])))

print("Большое количество деталей")
display(HTML(''.join([make_html(x) for x in _group_3_paths])))

Малое количество деталей


Средне количество деталей


Большое количество деталей


## Вносим искажения в изображения 

In [2]:
import cv2
import os
import numpy as np

# Сохраняем изображения с потерями
def save_low_quality(in_path):
    _QUALITY = 1
    _dirname = os.path.dirname(in_path)
    _filename = os.path.basename(in_path).replace(".jpg", "")
    _result_dirname = os.path.join(_dirname, "results")
    _result_path = os.path.join(_result_dirname, _filename + f"_low_quality_{_QUALITY}" + ".jpg")
    img = cv2.imread(in_path)
    cv2.imwrite(_result_path, img, [cv2.IMWRITE_JPEG_QUALITY, _QUALITY])

print("Сохраняем изображения с потерями")
[save_low_quality(x) for x in _group_1_paths]
[save_low_quality(x) for x in _group_2_paths]
[save_low_quality(x) for x in _group_3_paths]

# Нанесение размытия (blur)
def save_blur(in_path):
    _dirname = os.path.dirname(in_path)
    _filename = os.path.basename(in_path).replace(".jpg", "")
    _result_dirname = os.path.join(_dirname, "results")
    _result_path = os.path.join(_result_dirname, _filename + "_blur" + ".jpg")
    img = cv2.imread(in_path)
    blur_img = cv2.blur(img, (7, 7))
    cv2.imwrite(_result_path, blur_img)

print("Размываем изображения ядром 7х7 (blur)")
[save_blur(x) for x in _group_1_paths]
[save_blur(x) for x in _group_2_paths]
[save_blur(x) for x in _group_3_paths]

# Добавляем импульсный шум
def save_impulse_noise(in_path):
    noise_probability = 50
    _dirname = os.path.dirname(in_path)
    _filename = os.path.basename(in_path).replace(".jpg", "")
    _result_dirname = os.path.join(_dirname, "results")
    _result_path = os.path.join(_result_dirname, _filename + "_impulse_noise" + ".jpg")
    img = cv2.imread(in_path)
    h, w = img.shape[:2]
    noise = np.zeros((h, w), np.uint8)
    cv2.randu(noise, 0, 255)
    image_with_noise = img.copy()
    image_with_noise[noise < noise_probability] = 0
    image_with_noise[noise > 255 - noise_probability] = 255
    cv2.imwrite(_result_path, image_with_noise)

print("Добавляем импульсный шум")
[save_impulse_noise(x) for x in _group_1_paths]
[save_impulse_noise(x) for x in _group_2_paths]
[save_impulse_noise(x) for x in _group_3_paths]

print("**************************************")



Сохраняем изображения с потерями
Размываем изображения ядром 7х7 (blur)
Добавляем импульсный шум
**************************************


## Расчитываем три показателя: MSE, PSNR и SSIM

In [4]:
# python -m pip install -U scikit-image
# pip install pandas
# pip install openpyxl
import cv2
import os
import numpy as np
import pandas as pd
from skimage.metrics import peak_signal_noise_ratio as psnr
from skimage.metrics import structural_similarity as ssim
from skimage.metrics import mean_squared_error as mse

_group_1_paths = ['_images/group_1/_1.jpg', '_images/group_1/_2.jpg', '_images/group_1/_3.jpg']
_group_2_paths = ['_images/group_2/_1.jpg', '_images/group_2/_2.jpg', '_images/group_2/_3.jpg']
_group_3_paths = ['_images/group_3/_1.jpg', '_images/group_3/_2.jpg', '_images/group_3/_3.jpg']

df = pd.DataFrame()
def Calc_MSE_PSNR_SSIM(orig_img_path, name_group):
    global df
    orig_img = cv2.imread(orig_img_path)
    _basename = os.path.basename(orig_img_path).replace(".jpg", "")
    _dirname = os.path.dirname(orig_img_path)
    _result_dirname = os.path.join(_dirname, "results")
    for root, dirs, files in os.walk(_result_dirname):
        for file in files:
            if _basename not in os.path.basename(file)[:5]: continue; 
            dif_img  = cv2.imread(os.path.join(root, file))
            _MSE = mse(orig_img, dif_img)
            _PSNR = psnr(orig_img, dif_img)
            _SSIM = ssim(orig_img, dif_img, channel_axis = -1)
            #print(f"----- {_dirname}: {os.path.basename(orig_img_path)} -> {os.path.basename(file)} -----")
            #print(f'MSE: {round(_MSE, 2)}, PSNR: {round(_PSNR, 2)}, SSIM: {round(_SSIM, 2)}')
            df = df.append({'Group': name_group, 'BaseFile': os.path.basename(orig_img_path), 'DifFile': os.path.basename(file), 'MSE': round(_MSE, 3), 'PSNR': round(_PSNR, 3), 'SSIM': round(_SSIM, 3)}, ignore_index=True)

print("**************************************")
[Calc_MSE_PSNR_SSIM(x, "Small") for x in _group_1_paths]
print("**************************************")
[Calc_MSE_PSNR_SSIM(x, "Middle") for x in _group_2_paths]
print("**************************************")
[Calc_MSE_PSNR_SSIM(x, "Large") for x in _group_3_paths]
print("**************************************")

print(df)
df.to_excel("_images/output_Calc_MSE_PSNR_SSIM.xlsx")

**************************************


C:\Users\Acer\AppData\Local\Temp\ipykernel_9340\2822046262.py:32: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append({'Group': name_group, 'BaseFile': os.path.basename(orig_img_path), 'DifFile': os.path.basename(file), 'MSE': round(_MSE, 3), 'PSNR': round(_PSNR, 3), 'SSIM': round(_SSIM, 3)}, ignore_index=True)
C:\Users\Acer\AppData\Local\Temp\ipykernel_9340\2822046262.py:32: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append({'Group': name_group, 'BaseFile': os.path.basename(orig_img_path), 'DifFile': os.path.basename(file), 'MSE': round(_MSE, 3), 'PSNR': round(_PSNR, 3), 'SSIM': round(_SSIM, 3)}, ignore_index=True)
C:\Users\Acer\AppData\Local\Temp\ipykernel_9340\2822046262.py:32: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pa

**************************************


C:\Users\Acer\AppData\Local\Temp\ipykernel_9340\2822046262.py:32: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append({'Group': name_group, 'BaseFile': os.path.basename(orig_img_path), 'DifFile': os.path.basename(file), 'MSE': round(_MSE, 3), 'PSNR': round(_PSNR, 3), 'SSIM': round(_SSIM, 3)}, ignore_index=True)
C:\Users\Acer\AppData\Local\Temp\ipykernel_9340\2822046262.py:32: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append({'Group': name_group, 'BaseFile': os.path.basename(orig_img_path), 'DifFile': os.path.basename(file), 'MSE': round(_MSE, 3), 'PSNR': round(_PSNR, 3), 'SSIM': round(_SSIM, 3)}, ignore_index=True)
C:\Users\Acer\AppData\Local\Temp\ipykernel_9340\2822046262.py:32: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pa

**************************************


C:\Users\Acer\AppData\Local\Temp\ipykernel_9340\2822046262.py:32: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append({'Group': name_group, 'BaseFile': os.path.basename(orig_img_path), 'DifFile': os.path.basename(file), 'MSE': round(_MSE, 3), 'PSNR': round(_PSNR, 3), 'SSIM': round(_SSIM, 3)}, ignore_index=True)
C:\Users\Acer\AppData\Local\Temp\ipykernel_9340\2822046262.py:32: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append({'Group': name_group, 'BaseFile': os.path.basename(orig_img_path), 'DifFile': os.path.basename(file), 'MSE': round(_MSE, 3), 'PSNR': round(_PSNR, 3), 'SSIM': round(_SSIM, 3)}, ignore_index=True)
C:\Users\Acer\AppData\Local\Temp\ipykernel_9340\2822046262.py:32: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pa

**************************************
     Group BaseFile               DifFile        MSE    PSNR   SSIM
0    Small   _1.jpg           _1_blur.jpg    221.086  24.685  0.913
1    Small   _1.jpg  _1_impulse_noise.jpg  10862.836   7.771  0.045
2    Small   _1.jpg  _1_low_quality_1.jpg    254.594  24.072  0.759
3    Small   _2.jpg           _2_blur.jpg     66.296  29.916  0.897
4    Small   _2.jpg  _2_impulse_noise.jpg   6886.964   9.751  0.026
5    Small   _2.jpg  _2_low_quality_1.jpg    347.115  22.726  0.749
6    Small   _3.jpg           _3_blur.jpg     11.024  37.707  0.973
7    Small   _3.jpg  _3_impulse_noise.jpg   6957.270   9.706  0.013
8    Small   _3.jpg  _3_low_quality_1.jpg    314.338  23.157  0.865
9   Middle   _1.jpg           _1_blur.jpg    362.640  22.536  0.589
10  Middle   _1.jpg  _1_impulse_noise.jpg   7657.863   9.290  0.083
11  Middle   _1.jpg  _1_low_quality_1.jpg    604.980  20.313  0.551
12  Middle   _2.jpg           _2_blur.jpg    103.334  27.988  0.723
13  Middl